<br><br>
# <strong>Deep Q Network
<strong>강화학습 알고리즘인 Q-learning</strong>을 <strong>Deep Learning</strong> 으로 구현한 모듈

<br>
## <strong>1 Q - Learning 개요
1. <strong>게임 화면 영상</strong>을 통해서 학습하므로, <strong>Input은 CNN 신경망</strong>을 사용한다
1. Q-learning을 신경망으로 구현하면 불안정 하다고 알려졌지만, <strong>Deep Mind</strong>에서 이를 해결
    1. <strong>과거의 상태를 기억</strong>한 뒤, <strong>임의의 상태값</strong>을 추출하여 학습하면 <strong>특수한 상황에 치우치지 않는 학습</strong>이 가능하다
    1. 손실값 계산시 최적의 행동을 얻는 <strong>기본신경망</strong>과, 얻어낸 값이 좋은 선택인지를 비교하는 <strong>목표신경망</strong>을 분리하여 학습한
    다
1. DQN 기본 개념
    1. <strong>Replay Memory(메모리)</strong>에서 무작위 샘플링을 한다
    1. <strong>Q-net(기본신경망), Target Q-net(목표신경망)</strong> 간에는 일정시간마다 학습결과를 복사한다
    1. <strong>DQN-loss</strong>(손실값 계산)을 활용하여 학습결과를 갱신한다

<img src="http://www.samvitjain.com/blog/assets/massively-parallel-methods/Gorila-architecture.png" align='left' width='700'>

### <strong>01 Q 함수 (통계학의 이론)</strong>
1. <strong>Q-learning</strong> : 어떤 상태에서 <strong>특정행동의 가치</strong>를 나타내는 <strong>Q함수</strong>를 학습하는 알고리즘
1. <strong>Q-함수</strong> : 정규분포의 <strong>기각역(꼬리부분)의 확률 적분</strong>을 쉽게 구하는 함수이다 (아랫그림)
    
<img src="https://qph.ec.quoracdn.net/main-qimg-49e1002a0307a9ee73e679644a2fd971" align='left' width='250'>

### <strong>02 게임의 소개</strong>
OpenAI 비영리회사에서 제공한 모듈을 사용 
1. https://github.com/openai/gym : Atari 게임을 학습 (컴퓨팅 파워가 많이 소요되고, 시간도 오래 걸린다)
1. https://github.com/golbin/TensorFlow-Tutorials/tree/master/12%20-%20DQN : 간단한 박스 쌓기게임으로 Q-net을 구현

<br>
## <strong>2 에이전트 구현하기</strong>
1. <strong>에이전트</strong>는 게임상태를 입력받아 <strong>신경망에 전달</strong>하고, 신경망에서 판단한 행동을 <strong>게임에 적용</strong>하며 단계를 진행하는 모듈이다
    1. <strong>학습모드 (Train)</strong> : 게임을 보여주진 않은 채 <strong>빠른 학습</strong>을 진행한다 
    1. <strong>실행모드 (replay)</strong> : <strong>학습한 결과를 이용</strong>해 플레이 하는 화면을 출력한다
1. <strong>import DQN</strong> : 신경망을 구현한 모듈이다

In [1]:
import tensorflow as tf
import numpy as np
import random, time, sys

from DQN.game import Game
from DQN.model import DQN

/home/markbaum/Python/python36/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### <strong>01 에이전트 파라미터 설정
각 모듈별로 파라미터를 선언한다

In [2]:
# tf.app.flags : 에이전트 실행시 모드를 나누어서 실행하도록 옵션을 설정한다
tf.app.flags.DEFINE_boolean("train", False, "학습모드 (게임을 보여주지 않습니다)")
FLAGS = tf.app.flags.FLAGS

In [3]:
# Q-net 학습모델의 하이퍼 파라미터를 설정
MAX_EPISODE = 10000           # 최대로 학습할 게임의 횟수
TARGET_UPDATE_INTERVAL = 1000 # 1000번 마다 신경망을 갱신한다 (Q-net(기본신경망), Target Q-net(목표신경망) 분리된 신경망을 갱신하는 간격)
TRAIN_INTERVAL = 4            # 게임 4프레임(상태)마다 한 번씩 학습을 한다 (Q-net)
OBSERVE = 100                 # 일정수준까지는 학습을 지켜보며 데이터를 축적한다

In [4]:
# 게임 플레이의 Control 파라미터를 설정
NUM_ACTION = 3   # 행동내용 :  { 0 : 왼쪽이동 , 1 : stay , 2 : 오른쪽 이동 }
SCREEN_WIDTH = 6
SCREEN_HEIGHT = 10

### <strong>02 학습 모듈 정의하기
학습 부분의 모듈 설정

In [5]:
def train():
    print('뇌세포 깨우는 중..')
    sess = tf.Session()
    game = Game(SCREEN_WIDTH, SCREEN_HEIGHT, show_game=False)
    brain = DQN(sess, SCREEN_WIDTH, SCREEN_HEIGHT, NUM_ACTION)
    rewards = tf.placeholder(tf.float32, [None])
    tf.summary.scalar('avg.reward/ep.', tf.reduce_mean(rewards))

    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('logs', sess.graph)
    summary_merged = tf.summary.merge_all()

    brain.update_target_network()             # 타겟 네트웍을 초기화
    epsilon = 1.0  # 다음에 취할 액션을 DQN 을 이용해 결정할 시기를 결정
    time_step = 0  # 프레임 횟수
    total_reward_list = []

    for episode in range(MAX_EPISODE): # 게임을 시작
        terminal = False
        total_reward = 0
        # 상태는 screen_width x screen_height 크기의 화면 구성을 의미한다
        state = game.reset()     # 게임을 초기화하고 현재 상태를 가져온다
        brain.init_state(state)
        while not terminal:
            if np.random.rand() < epsilon:  # 1.입실론(임의 작은양수)보다 작을땐 랜덤한 액션 실행
                action = random.randrange(NUM_ACTION)
            else:                           # 2.랜덤값이 그 이상일 땐 DQN 액션을 선택
                action = brain.get_action()
            if episode > OBSERVE:           # 일정횟수 뒤 부터는 입실론 값을 줄여서 DQN 학습을 자주 실행
                epsilon -= 1 / 1000

            # 액션을 이용해 게임을 진행하고, 보상과 게임의 종료 여부를 받아온디.
            state, reward, terminal = game.step(action)
            total_reward += reward

            # 현재 상태를 Brain에 기억 : 기억을 이용해 학습, 다음 행동을 결정
            brain.remember(state, action, reward, terminal)
            if time_step > OBSERVE and time_step % TRAIN_INTERVAL == 0:
                brain.train()                      # DQN 으로 학습을 진행
            if time_step % TARGET_UPDATE_INTERVAL == 0: 
                brain.update_target_network()     # 타겟 네트웍을 업데이트
            time_step += 1

        print('게임횟수: %d 점수: %d' % (episode + 1, total_reward))
        total_reward_list.append(total_reward)
        if episode % 10 == 0:
            summary = sess.run(summary_merged, feed_dict={rewards: total_reward_list})
            writer.add_summary(summary, time_step)
            total_reward_list = []

        if episode % 100 == 0:
            saver.save(sess, 'model/dqn.ckpt', global_step=time_step)

In [6]:
def replay():
    print('뇌세포 깨우는 중..')
    sess = tf.Session()
    game = Game(SCREEN_WIDTH, SCREEN_HEIGHT, show_game=True)
    brain = DQN(sess, SCREEN_WIDTH, SCREEN_HEIGHT, NUM_ACTION)

    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state('model')
    saver.restore(sess, ckpt.model_checkpoint_path)
    for episode in range(MAX_EPISODE):  # 게임을 시작
        terminal = False
        total_reward = 0
        state = game.reset()
        brain.init_state(state)
        # 학습한 액션을 이용해 게임을 진행, 보상과 게임의 종료 여부를 받아온다
        while not terminal:
            action = brain.get_action()
            state, reward, terminal = game.step(action)
            total_reward += reward
            brain.remember(state, action, reward, terminal)
            time.sleep(0.3) # 인지 가능한 속도로 연산을 시행한다
        print('게임횟수: %d 점수: %d' % (episode + 1, total_reward))

In [7]:
def main(_):
    if FLAGS.train:
        train()
    else:
        replay()

In [8]:
# tf.app.run()   # Jupyter 에서 실행하면 오류가 발생, 커널에서 실행시 잘 작동한다

<br>
## <strong>3 신경망 모델 구현하기</strong>
<strong>DQN</strong>의 구현

In [9]:
# 알파고를 만든 구글의 딥마인드의 논문을 참고한 DQN 모델을 생성합니다.
# http://www.nature.com/nature/journal/v518/n7540/full/nature14236.html
import tensorflow as tf
import numpy as np
import random
from collections import deque

In [10]:
class DQN:
    REPLAY_MEMORY = 10000  # '학습용 플레이결과' 저장 숫자를 정의 : (플레이결과 = 게임판의 상태 + 취한 액션 + 리워드 + 종료여부)
    BATCH_SIZE = 32  # 학습시 사용할 replay memory의 갯수를 정의
    GAMMA = 0.99  # 과거상태 가중치를 줄이는 매개변수
    STATE_LEN = 4 # 한 번에 볼 총 프레임 수 (앞의 상태까지 고려)
    def __init__(self, session, width, height, n_action):
        self.session = session
        self.n_action = n_action
        self.width = width
        self.height = height
        self.memory = deque()  # 게임 플레이결과를 저장할 메모리
        self.state = None      # 현재 게임판의 상태
        # 게임의 상태 : [게임판의 가로 크기, 게임판의 세로 크기, 게임 상태의 갯수(현재+과거+과거..)]
        self.input_X = tf.placeholder(tf.float32, [None, width, height, self.STATE_LEN])
        self.input_A = tf.placeholder(tf.int64, [None])    # 개별상태 액션값 ex) 0, 1, 0, 1, 2, 1, ..
        self.input_Y = tf.placeholder(tf.float32, [None])  # 손실값 계산시 사용하는 입력값
        self.Q = self._build_network('main')
        self.cost, self.train_op = self._build_op()
        # 학습을 효율을 높이기 위해, 손실 타겟(실측값)의 Q value를 계산하는 네트웍을 따로 만들어서 사용
        self.target_Q = self._build_network('target')

    def _build_network(self, name):
        with tf.variable_scope(name):
            model = tf.layers.conv2d(self.input_X, 32, [4, 4], padding='same', activation=tf.nn.relu)
            model = tf.layers.conv2d(model, 64, [2, 2], padding='same', activation=tf.nn.relu)
            model = tf.contrib.layers.flatten(model)
            model = tf.layers.dense(model, 512, activation=tf.nn.relu)
            Q = tf.layers.dense(model, self.n_action, activation=None)
        return Q

    def _build_op(self): # DQN 의 손실 함수를 정의 : gradient descent step on (y_j-Q(ð_j,a_j;θ))^2
        one_hot = tf.one_hot(self.input_A, self.n_action, 1.0, 0.0)
        Q_value = tf.reduce_sum(tf.multiply(self.Q, one_hot), axis=1)
        cost = tf.reduce_mean(tf.square(self.input_Y - Q_value))
        train_op = tf.train.AdamOptimizer(1e-6).minimize(cost)
        return cost, train_op

    def update_target_network(self):  # refer: https://github.com/hunkim/ReinforcementZeroToAll/
        copy_op = []
        main_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='main')
        target_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='target')
        # '학습네트웍'의 변수값을 '타겟네트웍'으로 업데이트
        for main_var, target_var in zip(main_vars, target_vars):
            copy_op.append(target_var.assign(main_var.value()))
        self.session.run(copy_op)

    def get_action(self):
        Q_value = self.session.run(self.Q,
                                   feed_dict={self.input_X: [self.state]})
        action = np.argmax(Q_value[0])
        return action

    def init_state(self, state): # 게임판을 초기화 한다. (앞의 상태까지 고려)
        state = [state for _ in range(self.STATE_LEN)]
        # axis=2 : input_X 이 '컨볼루션 레이어' 마지막 차원을 쌓아올린 형태로 정의되었기 때문
        self.state = np.stack(state, axis=2)

    def remember(self, state, action, reward, terminal):
        # 과거상태까지 고려하여 학습을 계산한다 (과거 3번 + 현재 = 총 4번의 상태를 계산)
        # 새로운 상태가 입력시, 가장 오래된 상태를 제거하고 새로운 상태로 대체한다
        next_state = np.reshape(state, (self.width, self.height, 1))
        next_state = np.append(self.state[:, :, 1:], next_state, axis=2)
        # 액션결과, 상태값과 보상을 메모리에 저장
        self.memory.append((self.state, next_state, action, reward, terminal))
        # 저장할 플레이결과의 갯수를 제한
        if len(self.memory) > self.REPLAY_MEMORY:
            self.memory.popleft()
        self.state = next_state

    def _sample_memory(self):
        sample_memory = random.sample(self.memory, self.BATCH_SIZE)
        state = [memory[0] for memory in sample_memory]
        next_state = [memory[1] for memory in sample_memory]
        action = [memory[2] for memory in sample_memory]
        reward = [memory[3] for memory in sample_memory]
        terminal = [memory[4] for memory in sample_memory]
        return state, next_state, action, reward, terminal

    def train(self):
        # 저장 메모리에서 '배치사이즈'만큼 샘플을 추출
        state, next_state, action, reward, terminal = self._sample_memory()
        # 학습시 다음 상태를 타겟 네트웍에 넣어 target Q value를 계산
        target_Q_value = self.session.run(self.target_Q,
                                          feed_dict={self.input_X: next_state})
        # input_Y 에 들어갈 값들을 계산한다 : DQN의 손실함수의 핵심적인 값을 계산
        # if episode is terminates at step j+1 then r_j  \\ otherwise r_j + γ*max_a'Q(ð_(j+1),a';θ')
        Y = []
        for i in range(self.BATCH_SIZE):
            if terminal[i]:   Y.append(reward[i])
            else:             Y.append(reward[i] + self.GAMMA * np.max(target_Q_value[i]))
        self.session.run(self.train_op,
                         feed_dict={self.input_X: state, self.input_A: action, self.input_Y: Y})

<br>
## <strong>3 학습 모델의 구현
<strong>terminal $</strong> python agent.py --train